In [1]:
MARKET = 'SNP'

In [2]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [3]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder

In [4]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = DATAPATH.joinpath('temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [5]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [6]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

array(['df_chains.pkl', 'df_ohlcs.pkl', 'df_opt_margins.pkl',
       'df_opt_prices.pkl', 'df_symlots.pkl', 'df_und_margins.pkl',
       'df_unds.pkl', 'dfrq.pkl', 'qopts.pkl'], dtype='<U18')

# Running `Covers`

In [9]:
from dfrq import get_dfrq
from support import Timer

In [10]:
ibp = Vars(MARKET.upper())  # IB Parameters from var.yml

HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

LOGPATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", "log")
DATAPATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", MARKET.lower())

# * SETUP LOGS AND CLEAR THEM
LOGFILE = LOGPATH.joinpath(MARKET.lower() + "_cover.log")
util.logToFile(path=LOGFILE, level=30)
with open(LOGFILE, "w"):
    pass

# . start the time
cover_time = Timer('cover')
cover_time.start()
dfrq = get_dfrq(MARKET)


cover started at 20-Nov-2020 01:27:27

dfrqs started at 20-Nov-2020 01:27:27

Done margin for ['COP0115P50.0', 'DOW0115P40.0'] 19 out of 59. Pending ['AAPL1120P87.5', 'ALGN0.0']

...dfrqs took: 00:00:15 seconds



In [11]:
dfrq

,symbol,undPrice,lot,gross,grosspos,remq,status
13,NVDA,527.92,100,53715.0,147500.000000,0,harvest
49,AMGN,224.94,100,22776.0,92789.215090,0,uncovered
41,ADSK,256.54,100,25222.0,50000.000000,0,harvest
18,ALGN,444.65,100,44461.0,44047.735595,0,dodo
19,LRCX,430.75,100,43033.0,42940.753175,0,dodo
...,...,...,...,...,...,...,...
249,APA,11.25,100,1104.0,NaN,20,fresh
250,HPE,10.46,100,1050.0,NaN,21,fresh
251,GE,9.65,100,973.0,NaN,23,fresh
252,F,8.83,100,882.0,NaN,25,fresh


In [22]:
dfrq[dfrq.status.isin(['uncovered', 'dodo'])]

,symbol,undPrice,lot,gross,grosspos,remq,status
49,AMGN,224.94,100,22776.0,92789.215090,0,uncovered
18,ALGN,444.65,100,44461.0,44047.735595,0,dodo
19,LRCX,430.75,100,43033.0,42940.753175,0,dodo
27,UNH,338.58,100,34451.0,33484.817505,0,dodo
30,ANTM,316.50,100,32238.0,31193.167115,0,dodo
36,ANET,271.41,100,27203.0,27096.884155,0,dodo
100,ETSY,131.80,100,12701.0,26662.857060,0,dodo
72,UPS,167.41,100,16845.0,16619.400025,0,dodo
81,HSY,148.20,100,14984.0,14866.101075,0,dodo
151,RTX,70.06,100,6951.0,13970.999910,1,uncovered


In [84]:
from support import quick_pf, get_dte
from engine import executeAsync, save_df, price

with IB().connect(HOST, PORT, CID) as ib:
    df_pf = quick_pf(ib)

In [30]:
# ASSEMBLE STOCKS NEEDING COVER
# ... get the stocks and options
df_stk = df_pf[df_pf.secType == 'STK']
df_opt = df_pf[df_pf.secType == 'OPT']

# ... Filter out the partials
df_stk = df_stk[df_stk.position % 100 == 0]

# ... Filter out already covered and special stocks

#     ...from portfolio get symbol list of already existing covered calls
m_covered = (df_opt.position < 0) & df_opt.symbol.isin(df_stk.symbol)
already_covered = df_opt[m_covered].symbol.to_list()

df_uncovered = df_stk[~df_stk.symbol.isin(already_covered)]

In [60]:
# BUILD UP OPTIONS FOR THESE SYMBOLS

opts = [q for q in qopts if q.symbol in df_uncovered.symbol.unique()]
df_opts = util.df(opts)

In [73]:
optcols = "conId,symbol,secType,lastTradeDateOrContractMonth,strike,right".split(",")
df_uc = df_opts[optcols].rename(columns={"lastTradeDateOrContractMonth": "expiry"})

# ... get next week's earliest dte
df_uc = df_uc.assign(dte=df_uc.expiry.apply(get_dte))
m = (df_uc.dte>5)
df_uc = df_uc[m][df_uc[m].dte == df_uc[m].groupby('symbol').dte.transform(min)].reset_index(drop=True)

In [87]:
%%time
with IB().connect(HOST, PORT, CID) as ib:
    und_cts = ib.qualifyContracts(*[Contract(conId=c) for c in df_uncovered.conId])
    df_und_price = ib.run(
                    executeAsync(
                    ib=ib,
                    algo=price,
                    cts=und_cts,
                    CONCURRENT=500,
                    TIMEOUT=15,
                    post_process=save_df,
                    **{'FILL_DELAY': 15},
            )
        )


Done price for [] 0 out of 15. Pending ['ALGN0.0', 'AMGN0.0']
Wall time: 16 s


In [89]:
df_uncovered

,secType,conId,symbol,expiry,strike,right,position,mktPrice,mktVal,avgCost,unPnL,rePnL
6,STK,11459264,ALGN,,0.0,0,-100.0,441.259827,-44125.98,439.990157,-126.97,0.0
9,STK,266145,AMGN,,0.0,0,200.0,224.233994,44846.80,240.000000,-3153.20,0.0
11,STK,153912055,ANET,,0.0,0,-100.0,271.225769,-27122.58,229.994798,-4123.10,0.0
12,STK,173962302,ANTM,,0.0,0,-100.0,313.343048,-31334.30,302.493196,-1084.99,0.0
13,STK,5407,CAH,,0.0,0,-100.0,52.080002,-5208.00,50.498765,-158.12,0.0
15,STK,5749,CL,,0.0,0,-100.0,85.080002,-8508.00,81.998069,-308.19,0.0
16,STK,13878278,CNC,,0.0,0,-100.0,63.552002,-6355.20,65.998422,244.64,0.0
23,STK,190480965,ETSY,,0.0,0,-200.0,134.283630,-26856.73,144.496688,2042.61,0.0
24,STK,98542021,EXPE,,0.0,0,-100.0,122.379997,-12238.00,119.997229,-238.28,0.0
28,STK,8199,HSY,,0.0,0,-100.0,148.786789,-14878.68,147.996610,-79.02,0.0


In [92]:
df_uc = df_uc.set_index('symbol')\
             .join(df_und_price.set_index('symbol')[['price', 'iv']])\
             .rename(columns={'iv': 'und_iv'})\
             .join(df_uncovered.set_index('symbol'[['position', 'avgCost', 'unPnL']]))\
             .reset_index(drop=True)

<>:4: SyntaxWarning: str indices must be integers or slices, not list; perhaps you missed a comma?
<>:4: SyntaxWarning: str indices must be integers or slices, not list; perhaps you missed a comma?
<ipython-input-92-d61fb632b840>:4: SyntaxWarning: str indices must be integers or slices, not list; perhaps you missed a comma?
  .join(df_uncovered.set_index('symbol'[['position', 'avgCost', 'unPnL']]))\


TypeError: string indices must be integers